In [1]:
import os
import pandas as pd
import numpy as np
import xgboost as xgb

In [2]:
!ls ../data/

sample_submission.csv			       test.csv
santander-customer-transaction-prediction.zip  train.csv


In [3]:
df = pd.read_csv("../data/train.csv")

In [4]:
for i,v in enumerate(df.dtypes):
    if v!=np.float64:
        print(i,v)

0 object
1 int64


In [5]:
df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [6]:
Y = df["target"]
X= df.drop(["ID_code","target"],axis=1)

In [7]:
y_test,y_train = Y[:20000],Y[20000:]
x_test,x_train = X[:20000],X[20000:]

In [8]:
print("X shape:",X.shape)
print("Y:",set(Y))
X.head()

X shape: (200000, 200)
Y: {0, 1}


,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [9]:
import matplotlib.pyplot as plt

In [15]:
xgmat = xgb.DMatrix( x_train, label=y_train, missing = -999.0)

/home/timruning/anaconda3/envs/tensorflow/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/timruning/anaconda3/envs/tensorflow/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [17]:
num_round = 120

In [16]:
# setup parameters for xgboost
param = {}
# use logistic regression loss, use raw prediction before logistic transformation
# since we only need the rank
param['objective'] = 'binary:logitraw'
# scale weight of positive examples
param['eta'] = 0.1
param['max_depth'] = 6
param['eval_metric'] = 'auc'
param['silent'] = 1
param['nthread'] = 16
plst = list(param.items())+[('eval_metric', 'ams@0.15')]
watchlist = [ (xgmat,'train') ]


In [19]:
gbm = xgb.train(plst, xgmat, num_round, watchlist)

[0]	train-auc:0.652821	train-ams@0.15:38.4051
[1]	train-auc:0.68004	train-ams@0.15:44.7299
[2]	train-auc:0.693187	train-ams@0.15:45.6464
[3]	train-auc:0.704043	train-ams@0.15:47.7225
[4]	train-auc:0.713219	train-ams@0.15:49.1402
[5]	train-auc:0.721954	train-ams@0.15:49.9016
[6]	train-auc:0.727955	train-ams@0.15:51.0678
[7]	train-auc:0.736207	train-ams@0.15:52.5765
[8]	train-auc:0.74235	train-ams@0.15:53.6036
[9]	train-auc:0.750569	train-ams@0.15:54.7669
[10]	train-auc:0.757095	train-ams@0.15:56.193
[11]	train-auc:0.763958	train-ams@0.15:58.1855
[12]	train-auc:0.766332	train-ams@0.15:58.8223
[13]	train-auc:0.7759	train-ams@0.15:60.7444
[14]	train-auc:0.778421	train-ams@0.15:61.74
[15]	train-auc:0.783894	train-ams@0.15:62.8206
[16]	train-auc:0.793743	train-ams@0.15:64.6503
[17]	train-auc:0.799147	train-ams@0.15:66.2188
[18]	train-auc:0.803384	train-ams@0.15:67.5527
[19]	train-auc:0.806422	train-ams@0.15:68.013
[20]	train-auc:0.810192	train-ams@0.15:69.0193
[21]	train-auc:0.811977	train-a

In [36]:
X_test = xgb.DMatrix(x_test)

In [61]:
y_pre = gbm.predict(X_test)

In [63]:
y_pre

array([-3.353394 , -2.0628173, -3.2278404, ..., -3.7249866, -3.0541909,
       -3.2806456], dtype=float32)

In [51]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score

In [52]:
mean_squared_error(y_test,y_pre)

7.731738758521038

In [53]:
roc_auc_score(y_test,y_pre)

0.8516074319772978

In [ ]:
params_sklearn = {
    'learning_rate':0.1,
    'max_bin':150,
    'num_leaves':32,    
    'max_depth':11,
    
    'reg_alpha':0.1,
    'reg_lambda':0.2,   
     
    'objective':'binary',
    'n_estimators':300,
    #'class_weight':weight
}

In [70]:
gbm1 = xgb.XGBClassifier(**params_sklearn).fit(x_train,y_train)

In [71]:
y_pre = gbm1.predict(x_test)

In [72]:
mean_squared_error(y_test,y_pre)

0.08165

In [73]:
roc_auc_score(y_test,y_pre)

0.609782473434761

In [69]:
sum(y_pre)

652